In [1]:
import pandas as pd
from pydantic import BaseModel
from pandantic import Pandantic
from typing import Optional

## Insights da análise rápida dos datasets
# Não é possível usar gênero para categorizar, pois só tem no dataset secundario, e só tem 5 generos
# Existe alguns outros datasets que tem dados como o genero.

class MainDataset(BaseModel):
    valence: float
    year: int
    acousticness: float
    artists: str
    danceability: float
    duration_ms: int
    energy: float
    explicit: bool
    id: str
    instrumentalness: float
    key: int
    liveness: float
    loudness: float
    mode: bool
    name: str
    popularity: int
    release_date: int|str # TODO: Talvez eu precise tratar pra transformar os anos em datas YYYY-MM-DD
    speechiness: float
    tempo: float

class SecondaryDataset(BaseModel):
    track_id: str
    artists: Optional[str] = None
    album_name: Optional[str] = None
    track_name: Optional[str] = None
    popularity: int
    duration_ms: int
    explicit: bool
    danceability: float
    energy: float
    key: int
    loudness: float
    mode: bool
    speechiness: float
    acousticness: float
    instrumentalness: float
    liveness: float
    valence: float
    tempo: float
    time_signature: int
    track_genre: str # Não posso usar porque tem só 5 opções
    

In [30]:
import ast
main_df = pd.read_csv('./spotify-main-dataset/data.csv')
secondary_df = pd.read_csv('./spotify-aux-dataset/dataset.csv')

def a(x):
    if x.artists:
        x['artists'] = ast.literal_eval(x.artists)
    return x

main_df.apply(a, axis=1)
main_df.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [3]:
secondary_df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [10]:
def convert_secondary_to_main(secondary_row: dict) -> dict:
    """
    Convert a secondary dataset row to main dataset schema
    """
    return {
        'valence': secondary_row['valence'],
        'year': None,  # Not available in secondary dataset
        'acousticness': secondary_row['acousticness'],
        'artists': secondary_row.get('artists', '').split(";"),
        'danceability': secondary_row['danceability'],
        'duration_ms': secondary_row['duration_ms'],
        'energy': secondary_row['energy'],
        'explicit': secondary_row['explicit'],
        'id': secondary_row['track_id'],
        'instrumentalness': secondary_row['instrumentalness'],
        'key': secondary_row['key'],
        'liveness': secondary_row['liveness'],
        'loudness': secondary_row['loudness'],
        'mode': secondary_row['mode'],
        'name': secondary_row.get('track_name', ''),
        'popularity': secondary_row['popularity'],
        'release_date': None,  # Not available in secondary dataset
        'speechiness': secondary_row['speechiness'],
        'tempo': secondary_row['tempo']
    }


In [49]:

def merge_datasets(main_df: pd.DataFrame, secondary_df: pd.DataFrame) -> pd.DataFrame:
    """
    Merge secondary dataset into main dataset, avoiding duplicates based on ID
    """
    # Create a copy of main_df to avoid modifying the original
    merged_df = main_df.copy()
    # Get existing IDs from main dataset
    existing_ids = set(merged_df['id'].values)
    
    # Process each row in secondary dataset
    new_rows = []
    for _, row in secondary_df.iterrows():
        # Check if track_id already exists in main dataset
        if row['track_id'] not in existing_ids:
            # Convert to main dataset schema
            converted_row = convert_secondary_to_main(row.to_dict())
            new_rows.append(converted_row)
            # Add to existing_ids to avoid duplicates within the secondary dataset
            existing_ids.add(row['track_id'])
    # Create DataFrame from new rows and append to merged_df
    if new_rows:
        new_df = pd.DataFrame(new_rows)
        merged_df = pd.concat([merged_df, new_df], ignore_index=True)
    
    return merged_df


In [55]:
# TODO: PENSAR SE DEVO REMOVER OS NOMES DE MÚSICAS DUPLICADAS OU NÃO, acho que não,
#  embora eu precise criar um "modelo" pra inferir que duas músicas são iguais pra não recomendar mais de 2 músicas iguais
df = merge_datasets(main_df, secondary_df)
# print(len(df))
# unique = df.drop_duplicates(subset=['id'])
# print(len(unique))
df.head()


253606
253606


,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [27]:
df.count()

valence             253606
year                170653
acousticness        253606
artists             253605
danceability        253606
duration_ms         253606
energy              253606
explicit            253606
id                  253606
instrumentalness    253606
key                 253606
liveness            253606
loudness            253606
mode                253606
name                253605
popularity          253606
release_date        170653
speechiness         253606
tempo               253606
dtype: int64

In [28]:
# df.to_csv('output_dataframe.csv', index=False)

In [ ]:
genres_df = pd.read_csv('./spotify-main-dataset/data_by_genres.csv')